- 라벨 인코딩한 범주형 변수들 타입을 category로 바꿔주기 : 똑같은
- 결과변수 category로 바꾸지 말고 해보기 : 똑같을듯
- 상관관계를 통해 변수 삭제

In [38]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action = 'ignore')

In [39]:
## 데이터 불러오기
trn = pd.read_csv('data/train.csv')
tst = pd.read_csv('data/test.csv')

## 변수 생성
- 2개의 파생변수만을 생성함

In [40]:
## 365 변환
def days_to_plus(x):
    return (x*-1)/365

## 업무 시작일 변환
def plus(x):
    if x<0:
        return x*(-1)
    else:
        return 0

## 나이 양수로 변환
trn['birth_plus'] = trn.DAYS_BIRTH.map(days_to_plus)
tst['birth_plus'] = tst.DAYS_BIRTH.map(days_to_plus)
    
## 업무시작일 양수로 변환
trn['employed_plus'] = trn.DAYS_EMPLOYED.map(days_to_plus)
tst['employed_plus'] = tst.DAYS_EMPLOYED.map(days_to_plus)

In [41]:
trn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          26457 non-null  int64  
 1   gender         26457 non-null  object 
 2   car            26457 non-null  object 
 3   reality        26457 non-null  object 
 4   child_num      26457 non-null  int64  
 5   income_total   26457 non-null  float64
 6   income_type    26457 non-null  object 
 7   edu_type       26457 non-null  object 
 8   family_type    26457 non-null  object 
 9   house_type     26457 non-null  object 
 10  DAYS_BIRTH     26457 non-null  int64  
 11  DAYS_EMPLOYED  26457 non-null  int64  
 12  FLAG_MOBIL     26457 non-null  int64  
 13  work_phone     26457 non-null  int64  
 14  phone          26457 non-null  int64  
 15  email          26457 non-null  int64  
 16  occyp_type     18286 non-null  object 
 17  family_size    26457 non-null  float64
 18  begin_

## Label Encoding

In [42]:
index_col = 'index'
target_col = 'credit'

cat_cols = [x for x in trn.columns if trn[x].dtype == 'object']
float_cols = [x for x in trn.columns.drop('credit') if trn[x].dtype == 'float64']
num_cols = [x for x in trn.columns if x not in cat_cols + [target_col]]
feature_cols = num_cols + cat_cols
print(len(feature_cols), len(cat_cols), len(num_cols))

21 8 13


In [43]:
from sklearn.preprocessing import LabelEncoder

lbe = LabelEncoder()
for i in cat_cols:
    trn[i] = lbe.fit_transform(trn[i])
    tst[i] = lbe.transform(tst[i])
    
for i in float_cols:
    trn[i] = trn[i].astype('int')
    tst[i] = tst[i].astype('int')

In [44]:
for i in cat_cols:
    trn[i] = trn[i].astype('category')
    tst[i] = tst[i].astype('category')
    
## 결과 변수 category 형으로 만들기
trn.credit = trn.credit.astype('category')

In [45]:
trn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   index          26457 non-null  int64   
 1   gender         26457 non-null  category
 2   car            26457 non-null  category
 3   reality        26457 non-null  category
 4   child_num      26457 non-null  int64   
 5   income_total   26457 non-null  int32   
 6   income_type    26457 non-null  category
 7   edu_type       26457 non-null  category
 8   family_type    26457 non-null  category
 9   house_type     26457 non-null  category
 10  DAYS_BIRTH     26457 non-null  int64   
 11  DAYS_EMPLOYED  26457 non-null  int64   
 12  FLAG_MOBIL     26457 non-null  int64   
 13  work_phone     26457 non-null  int64   
 14  phone          26457 non-null  int64   
 15  email          26457 non-null  int64   
 16  occyp_type     26457 non-null  category
 17  family_size    26457 non-null  

In [46]:
## 인덱스 설정
trn.set_index('index', inplace = True)
tst.set_index('index', inplace = True)

In [47]:
trn.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,...,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit,birth_plus,employed_plus
index,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,202500,0,1,1,2,-13899,...,1,0,0,0,18,2,-6,1.0,38,12
1,0,0,1,1,247500,0,4,0,1,-11380,...,1,0,0,1,8,3,-5,1.0,31,4
2,1,1,1,0,450000,4,1,1,1,-19087,...,1,0,1,0,10,2,-22,2.0,52,12
3,0,0,1,0,202500,0,4,1,1,-15088,...,1,0,1,0,14,2,-37,0.0,41,5
4,0,1,1,0,157500,2,1,1,1,-15037,...,1,0,0,0,10,2,-26,2.0,41,5


In [34]:
tst.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 26457 to 36456
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   gender         10000 non-null  int32
 1   car            10000 non-null  int32
 2   reality        10000 non-null  int32
 3   child_num      10000 non-null  int64
 4   income_total   10000 non-null  int32
 5   income_type    10000 non-null  int32
 6   edu_type       10000 non-null  int32
 7   family_type    10000 non-null  int32
 8   house_type     10000 non-null  int32
 9   DAYS_BIRTH     10000 non-null  int64
 10  DAYS_EMPLOYED  10000 non-null  int64
 11  FLAG_MOBIL     10000 non-null  int64
 12  work_phone     10000 non-null  int64
 13  phone          10000 non-null  int64
 14  email          10000 non-null  int64
 15  occyp_type     10000 non-null  int32
 16  family_size    10000 non-null  int32
 17  begin_month    10000 non-null  int32
 18  birth_plus     10000 non-null  int32
 19  

In [35]:
# train과 valid(test) 셋을 분리 
x_train, x_valid, y_train, y_valid = train_test_split(trn.drop('credit', axis=1), trn.credit, test_size=0.3, stratify=trn.credit, random_state=40)

# RF 모델링 
rf = RandomForestClassifier(n_estimators = 5000,
                            max_features = 3,
                            min_samples_split = 6,
                            random_state = 2021,
                            n_jobs = 1
                           )
rf.fit(x_train, y_train)

RandomForestClassifier(max_features=3, min_samples_split=6, n_estimators=5000,
                       n_jobs=1, random_state=2021)

In [36]:
rf_pred = rf.predict_proba(x_valid)

In [37]:
log_loss(y_valid, rf_pred)

0.7120824800381204

In [ ]:
import eli5 
from eli5.sklearn import PermutationImportance 

In [ ]:
perm = PermutationImportance(rf, 
                             scoring = 'neg_log_loss',
                             random_state = 2021).fit(x_valid, y_valid)

In [ ]:
eli5.show_weights(perm, top = x_valid.shape[1], feature_names = x_valid.columns.tolist())

## 벨리드 검정 기준
- 0.7163870280361224
: rf = RandomForestClassifier(n_estimators = 1000,
                           max_features = 3,
                            min_samples_split = 5,
                            random_state = 2021,
                            n_jobs = -1
                            
                            
- 0.7123578685866444
: rf = RandomForestClassifier(n_estimators = 1000,
                           max_features = 3,
                            min_samples_split = 6,
                            random_state = 2021,
                            n_jobs = -1
                           )
                           
- 0.7120824800381204
: rf = RandomForestClassifier(n_estimators = 5000,
                           max_features = 3,
                            min_samples_split = 6,
                            random_state = 2021,
                            n_jobs = -1
                           )

In [53]:
# 테스트 예측
x_train, x_valid, y_train, y_valid = train_test_split(trn.drop('credit', axis=1), trn.credit, test_size=0.3, stratify=trn.credit, random_state=40)

# RF 모델링 
rf = RandomForestClassifier(n_estimators = 5000,
                           max_features = 3,
                            min_samples_split = 5,
                            random_state = 2021,
                            n_jobs = -1
                           )
rf.fit(trn.drop('credit', axis=1), trn.credit)

KeyboardInterrupt: 

In [ ]:
rf_pred = rf.predict_proba(tst)

In [ ]:
rf_pred

## 제출하기

In [15]:
sub = pd.read_csv('data/sample_submission.csv', index_col = 0)
sub[sub.columns] = rf_pred
sub.head()

,0,1,2
index,,,
26457,0.060745,0.213474,0.725781
26458,0.333988,0.292378,0.373634
26459,0.034598,0.051495,0.913907
26460,0.076464,0.054618,0.868919
26461,0.072112,0.185330,0.742558


In [16]:
sub.to_csv('data/n_estimator 차이.csv')

## 교차 검증 사용

In [54]:
ftr = trn.drop(['credit'],axis=1).values
target = trn['credit'].values
tst_ar = tst.values
n_class = 3
n_fold = 3

In [55]:
print(tst_ar.shape)
print()
print(ftr.shape)

(10000, 20)

(26457, 20)


In [ ]:
## predict_rf
print(ftr.shape)
print()
print(tst_ar.shape)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=40)

rf_p_val = np.zeros((ftr.shape[0], n_class))
rf_p_tst = np.zeros((tst_ar.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr, target), 1):
    print(f'training model for CV #{i}')
    rf_clf = RandomForestClassifier(n_estimators = 5000, 
                                random_state=2021,
                                max_features = 3,
                                min_samples_split = 6,
                                n_jobs=6)
    rf_clf.fit(ftr[i_trn], target[i_trn])
    rf_p_val[i_val, :] = rf_clf.predict_proba(ftr[i_val])
    rf_p_tst += rf_clf.predict_proba(tst_ar) / n_fold
    
print(f'{log_loss(target, rf_p_val)}')
print(f'{confusion_matrix(target, np.argmax(rf_p_val, axis=1))}%')

(26457, 20)

(10000, 20)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4


- 0.7181191299537627
: 3fold
[[  564   510  2148]
 [  137  2700  3430]
 [  337   865 15766]]%
     rf_clf = RandomForestClassifier(n_estimators = 5000, 
                                random_state=2021,
                                max_features = 3,
                                min_samples_split = 6,
                                n_jobs=6)
                                
- 0.7027768293407574
: 5fold
0.7027768293407574
[[  656   527  2039]
 [  149  2864  3254]
 [  355   899 15714]]%
     rf_clf = RandomForestClassifier(n_estimators = 5000, 
                                random_state=2021,
                                max_features = 3,
                                min_samples_split = 6,
                                n_jobs=6)

In [41]:
sub = pd.read_csv('data/sample_submission.csv', index_col = 0)
sub
sub[sub.columns] = rf_p_tst

In [42]:
sub.to_csv('data/최고기록 CV=5.csv')

In [ ]:
plot_importance(rf_clf)